In [23]:

# from PyPDF2 import PdfReader
from fillpdf import fillpdfs
# from pdfrw import PdfReader
import re
import json
# import PyPDF2
# import pdfrw
import re
# import fitz  # PyMuPDF

from enum import Enum
import re
import PyPDF2

class FormType(Enum):
    W8BEN = "W-8BEN"
    W9 = "W-9"
    W8IN = "W-8IN"

class BaseForm:
    def __init__(self, form_type,pdf_path):
            self.form_type = form_type
            self.pdf_path=pdf_path
            
    def load_json_structure(class_name):
        json_filename = f"{class_name}.json"
        with open(json_filename, 'r') as json_file:
            return json.load(json_file)


    @staticmethod 
    def readpdf_fields(pdf_file_name):
   
        fields=fillpdfs.get_form_fields(pdf_file_name,sort=False)
        return fields


    @staticmethod
    def readpdf_fields_byPypdf(pdf_file_name):
        f = PyPDF2.PdfReader(pdf_file_name)
        fields = f.get_fields()
        fdfinfo = dict((k, v.get('/V', '')) for k, v in fields.items())
        return fdfinfo

    @staticmethod
    def readpdf_fields_by_pdfrw(pdf_file_name):
        pdf = pdfrw.PdfReader(pdf_file_name)
        # print(pdf.keys())
        # print(pdf.Info)
        # print(pdf.Root.keys())
        # field_index=1

        all_fields={}
        for page in pdf.pages:
            annotations = page['/Annots']
            if annotations is None:
                continue
            
            for annotation in annotations:
                if annotation['/Subtype']=='/Widget':
                    if annotation['/T']:
                        key = annotation['/T'].to_unicode()
                        value = annotation['/V']
                        all_fields[key] = value
                    
        return all_fields    
    
    @staticmethod
    def identify_form_type(pdf_path):
        form_type = None
    
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            
            for page in pdf_reader.pages:
                text = page.extract_text()
                
                # Define keywords for each form type
                form_keywords = {
                    "W-8BEN": ["W-8BEN", "foreign status"],
                    "W-9": ["W-9", "Taxpayer Identification Number"],
                    "W-8IN": ["W-8IN", "Chapter 4"]
                }
                
                for form_name, keywords in form_keywords.items():
                    found_all_keywords = all(re.search(keyword, text, re.IGNORECASE) for keyword in keywords)
                    if found_all_keywords:
                        form_type = form_name
                        break
        
        return form_type
         
            




In [9]:
import textract
def convertts(fname):
    text = textract.process(fname)
    return text #.decode('utf-8') 

form_text=convertts("w8_ben.pdf")
print (form_text.decode("UTF-8"))
# write to .txt

In [21]:
import PyPDF2
from fillpdf import fillpdfs

def has_form_fields1(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return pdf_reader.get_fields()
       
def has_form_fields(pdf_file_name  ):
    
        fields=fillpdfs.get_form_fields(pdf_file_name,sort=False)
        return fields
    
    
# Provide the path to your PDF file
pdf_path = 'input.pdf'

if has_form_fields(pdf_path):
    print("The PDF has form fields.")
else:
    print("The PDF does not have form fields.")


The PDF has form fields.


In [2]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
# Path to the Tesseract executable (change this to your Tesseract installation path)

def extract_text_from_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    extracted_text = ''

    for img in images:
        img_text = pytesseract.image_to_string(img,config='--psm 12 --oem 3')
        extracted_text += img_text + '\n'

    return extracted_text

# Path to the non-searchable PDF
input_pdf_path = 'w8_ben.pdf'
extracted_text = extract_text_from_pdf(input_pdf_path)

# Print or process the extracted text as needed
print(extracted_text)


rom W-8BEN | Certificate of Foreign Status of Beneficial Owner for United

States Tax Withholding and Reporting (Individuals)

(Rev. October 2021)

> For use by individuals. Entities must use Form W-8BEN-E.

OMB No. 1545-1621

> Go to www.irs.gov/FormW8BEN for instructions and the latest information.

Department of the Treasury

> Give this form to the withholding agent or payer. Do not send to the IRS.

Do NOT use this form if:

Instead, use Form:

* You are NOT an individual

a

ee

W-8BEN-E

You are a U.S. citizen or other U.S. person, including a resident alien individual

sees we

ee

w-9

+ Youre beni wr ing coe elect cmece wih conc ot tae ree itn te Ut Sine

(other than personal services) .

W-8ECI

© You are a beneficial owner who is receiving compensation for personal services performed in the United States .

8233 or W-4

W-8IMY

© You are a person acting as an intermediary .

Note: If you are resident in a FATCA partner jurisdiction (that is, a Model 1 IGA jurisdiction with 

In [26]:

def identify_form_type_pic_and_all(pdf_path):
    form_type = None

    with open(pdf_path, 'rb') as pdf_file:
        images = convert_from_path(pdf_path)
        extracted_text = ''

        for img in images:
            img_text = pytesseract.image_to_string(img,config='--psm 12 --oem 3')
            extracted_text += img_text + '\n'

        
            
            # Define keywords for each form type
            form_keywords = {
                "W-8BEN": ["W-8BEN", "foreign status"],
                "W-9": ["W-9", "Taxpayer Identification Number"],
                "W-8IN": ["W-8IN", "Chapter 4"]
            }
            
            for form_name, keywords in form_keywords.items():
                found_all_keywords = all(re.search(keyword, extracted_text, re.IGNORECASE) for keyword in keywords)
                if found_all_keywords:
                    form_type = form_name
                    break
    
    return form_type

def identify_form_type_searchable(pdf_path):
    form_type = None

    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        for page in pdf_reader.pages:
            text = page.extract_text()
            
            print(text)
            
            # Define keywords for each form type
            form_keywords = {
                "W-8BEN": ["W-8BEN", "foreign status"],
                "W-9": ["W-9", "Taxpayer Identification Number"],
                "W-8IN": ["W-8IN", "Chapter 4"]
            }
            
            for form_name, keywords in form_keywords.items():
                found_all_keywords = all(re.search(keyword, text, re.IGNORECASE) for keyword in keywords)
                if found_all_keywords:
                    form_type = form_name
                    break
    
    return form_type

print(identify_form_type('input.pdf'))

W-8BEN


In [84]:
class W8BENForm(BaseForm):
    
    def __init__(self,pdf_path):
        super().__init__(FormType.W8BEN,pdf_path)
        self.fields = BaseForm.readpdf_fields(pdf_path)
        # print(self.fields)
        
    def load_json_structure(class_name):
        json_filename = f"{class_name}.json"
        with open(json_filename, 'r') as json_file:
            return json.load(json_file)


    # Define the JSON structure for W-8BEN form fields
    w8ben_form_structure = load_json_structure("W8BENForm")

    
        
        
    def isvalid(self):
        for field_info in self.w8ben_form_structure:
            field_identifier = field_info["field_identifier"]
            is_mandatory = field_info["mandatory"]
            validation_type = field_info["validation_type"]

            field_value = self.fields.get(field_identifier)
            if is_mandatory and field_value is None:
                return False
        
            # if field_value is not None:
            #     if validation_type == "text":
            #         if not isinstance(field_value, str) or len(field_value.strip()) == 0:
            #             print(field_identifier,field_value,is_mandatory)
            #             print('false')
            #             return False
            #     elif validation_type == "boolean":
            #         if not isinstance(field_value, bool):
            #             return False
    
        return True


    

        
    def getpdffields(self):
        # w8ben_json_data = {
        #     self.getw8ben_field_names_mapping().get(identifier, identifier): value
        #     for identifier, value in self.fields.items()
        # }
        # return json.dumps(w8ben_json_data, indent=4)
        mapped_data = {}
    
        for field_info in self.w8ben_form_structure:
            field_identifier = field_info["field_identifier"]
            field_name = field_info["field_name"]
            
            if field_identifier in self.fields:
                mapped_data[field_name] = self.fields[field_identifier]
        
        return mapped_data
    
    def getFields(self):
        return self.fields
        
       


NameError: name 'BaseForm' is not defined

In [ ]:


pdf_path='input.pdf'
detected_form_type = BaseForm.identify_form_type(pdf_path)
if detected_form_type:
        print(f"Detected Form Type: {detected_form_type}")

# Perform actions based on the detected form type
if detected_form_type == "W-8BEN":
    # Handle W-8BEN form
    form=W8BENForm(pdf_path)
    
        

    pass
elif detected_form_type == "W-9":
    # Handle W-9 form
    form=W9Form(pdf_path)
    pass
elif detected_form_type == "W-8IN":
    # Handle W-8IN form
    form=W9Form(pdf_path)
    pass
else:
    print("Could not determine the form type.")
    exit -1

if(form.isvalid()):
        # print(form.getFields(),"\n\n\n")
        print(form.getpdffields())

In [54]:


from PyPDF2 import PdfReader
from pprint import pprint

pdf_file_name = 'input.pdf'

f = PdfReader(pdf_file_name)
fields = f.get_fields()
fdfinfo = dict((k, v.get('/V', '')) for k, v in fields.items())
pprint(fdfinfo)



# with open('test.csv', 'w') as f2:
#     for key in fdfinfo.keys():
#         if type(key)==type("string") and type(str(fdfinfo[key]))==type("string"):
#             f2.write('"'+key+'","'+fdfinfo[key]+'"\n')
            
            
            
  



{'Date[0]': '',
 'Page1[0]': '',
 'c1_01[0]': '/1',
 'c1_02[0]': '/1',
 'f_10[0]': '123124 343',
 'f_11[0]': 'ewrwer',
 'f_12[0]': '12/12/12',
 'f_13[0]': '',
 'f_14[0]': 'src',
 'f_15[0]': '5',
 'f_16[0]': 'sdfsdf',
 'f_17[0]': '',
 'f_18[0]': '',
 'f_1[0]': 'fdgfdg',
 'f_20[0]': '',
 'f_21[0]': 'Diii sh',
 'f_2[0]': 'israel',
 'f_3[0]': 'dfgfdg',
 'f_4[0]': '1234 dereh pt',
 'f_5[0]': 'israel',
 'f_6[0]': 'fdgfdg',
 'f_7[0]': 'dfgfdg',
 'f_8[0]': '',
 'f_9[0]': '1234-343',
 'topmostSubform[0]': ''}


In [50]:
from fillpdf import fillpdfs
input_pdf_path="input.pdf"
fields=fillpdfs.get_form_fields(input_pdf_path,sort=False,page_number=None)
fillpdfs.print_form_fields(input_pdf_path, sort=False, page_number=None)
# fields


{'FEFF0066005F0031005B0030005D': 'fdgfdg',
'FEFF0066005F0032005B0030005D': 'israel',
'FEFF0066005F0033005B0030005D': 'dfgfdg',
'FEFF0066005F0034005B0030005D': '1234 dereh pt',
'FEFF0066005F0035005B0030005D': 'israel',
'FEFF0066005F0036005B0030005D': 'fdgfdg',
'FEFF0066005F0037005B0030005D': 'dfgfdg',
'FEFF0066005F0038005B0030005D': '',
'FEFF0066005F0039005B0030005D': '1234-343',
'FEFF0066005F00310030005B0030005D': '123124 343',
'FEFF00630031005F00300031005B0030005D': '1',
'FEFF0066005F00310031005B0030005D': 'ewrwer',
'FEFF0066005F00310032005B0030005D': '12/12/12',
'FEFF0066005F00310033005B0030005D': '',
'FEFF0066005F00310034005B0030005D': 'src',
'FEFF0066005F00310035005B0030005D': '5',
'FEFF0066005F00310036005B0030005D': 'sdfsdf',
'FEFF0066005F00310037005B0030005D': '',
'FEFF0066005F00310038005B0030005D': '',
'FEFF00630031005F00300032005B0030005D': '1',
'FEFF0066005F00320030005B0030005D': '',
'FEFF0044006100740065005B0030005D': '',
'FEFF0066005F00320031005B0030005D': 'Diii sh'}


In [56]:
import pdfrw

ANNOT_KEY = "/Annots"
ANNOT_FIELD_KEY = "/T"
ANNOT_VAL_KEY = "/V"
SUBTYPE_KEY = "/Subtype"
WIDGET_SUBTYPE_KEY = "/Widget"

PDF_NAME = "input.pdf"

template_pdf = pdfrw.PdfReader(PDF_NAME)
for page in range(0, len(template_pdf.pages)):
    annotations = template_pdf.pages[page][ANNOT_KEY]
    for annotation in annotations:
        if annotation[SUBTYPE_KEY] == WIDGET_SUBTYPE_KEY:
            if annotation[ANNOT_FIELD_KEY]:
                name = annotation[ANNOT_FIELD_KEY]

                print("{} ".format(name), end="")
                if annotation[ANNOT_VAL_KEY]:
                    value = annotation[ANNOT_VAL_KEY]
                    print("= {}".format(value))
                else:
                    print()

{'/AP': {'/N': (358, 0)}, '/DA': '(/HelveticaLTStd-Bold 8.00 Tf 0.000 0.000 0.502 rg)', '/F': '4', '/FT': '/Tx', '/Ff': '8388608', '/MK': {}, '/P': {'/Annots': [{...}, {'/AP': {'/N': (361, 0)}, '/DA': '(/HelveticaLTStd-Bold 8.00 Tf 0.000 0.000 0.502 rg)', '/F': '4', '/FT': '/Tx', '/Ff': '8388608', '/MK': {}, '/P': {...}, '/Parent': (18, 0), '/Q': '0', '/Rect': ['376.4', '540.008', '576', '554.009'], '/StructParent': '2', '/Subtype': '/Widget', '/T': '<FEFF0066005F0032005B0030005D>', '/Type': '/Annot', '/V': '(israel)'}, {'/AP': {'/N': (364, 0)}, '/DA': '(/HelveticaLTStd-Bold 8.00 Tf 0.000 0.000 0.502 rg)', '/F': '4', '/FT': '/Tx', '/Ff': '8388608', '/MK': {}, '/P': {...}, '/Parent': (18, 0), '/Q': '0', '/Rect': ['36', '516.007', '576', '530.008'], '/StructParent': '3', '/Subtype': '/Widget', '/T': '<FEFF0066005F0033005B0030005D>', '/Type': '/Annot', '/V': '(dfgfdg)'}, {'/AP': {'/N': (367, 0)}, '/DA': '(/HelveticaLTStd-Bold 8.00 Tf 0.000 0.000 0.502 rg)', '/F': '4', '/FT': '/Tx', '/Ff':

In [62]:
from pdfrw import PdfReader
input_pdf_path="input.pdf"
pdf = PdfReader(input_pdf_path)
# print(pdf.keys())
# print(pdf.Info)
# print(pdf.Root.keys())
# field_index=1

for page in pdf.pages:
    annotations = page['/Annots']
    if annotations is None:
        continue

    for annotation in annotations:
        if annotation['/Subtype']=='/Widget':
            if annotation['/T']:
                key = annotation['/T'].to_unicode()
                value = annotation['/V']
                print (key,':',value)
                
                
# print (pdf.Root.AcroForm.Fields[field_index].T)
# print('PDF has {} pages'.format(len(pdf.pages)))

f_1[0] : (fdgfdg)
f_2[0] : (israel)
f_3[0] : (dfgfdg)
f_4[0] : (1234 dereh pt)
f_5[0] : (israel)
f_6[0] : (fdgfdg)
f_7[0] : (dfgfdg)
f_8[0] : None
f_9[0] : (1234-343)
f_10[0] : (123124 343)
c1_01[0] : /1
f_11[0] : (ewrwer)
f_12[0] : (12/12/12)
f_13[0] : None
f_14[0] : (src)
f_15[0] : (5)
f_16[0] : (sdfsdf)
f_17[0] : None
f_18[0] : None
c1_02[0] : /1
f_20[0] : None
Date[0] : None
f_21[0] : (Diii sh)


In [69]:
from PyPDF3 import PdfFileWriter, PdfFileReader
# Open the PDF file
pdf_file = open('input.pdf', 'rb')
pdf_reader = PdfFileReader(pdf_file)
dictionary = pdf_reader.getFormTextFields() # returns a python dictionary
print(dictionary)
print(pdf_reader.getFields())
  
# Extract text data from each page
for page_num in range(pdf_reader.numPages):
    page = pdf_reader.getPage(page_num)
'   XXX' in page.extractText()
    

{'f_1[0]': 'fdgfdg', 'f_2[0]': 'israel', 'f_3[0]': 'dfgfdg', 'f_4[0]': '1234 dereh pt', 'f_5[0]': 'israel', 'f_6[0]': 'fdgfdg', 'f_7[0]': 'dfgfdg', 'f_8[0]': None, 'f_9[0]': '1234-343', 'f_10[0]': '123124 343', 'f_11[0]': 'ewrwer', 'f_12[0]': '12/12/12', 'f_13[0]': None, 'f_14[0]': 'src', 'f_15[0]': '5', 'f_16[0]': 'sdfsdf', 'f_17[0]': None, 'f_18[0]': None, 'Date[0]': None, 'f_21[0]': 'Diii sh'}
{'f_1[0]': {'/FT': '/Tx', '/Parent': {'/Kids': [IndirectObject(302, 0), IndirectObject(303, 0), IndirectObject(304, 0), IndirectObject(305, 0), IndirectObject(306, 0), IndirectObject(307, 0), IndirectObject(308, 0), IndirectObject(309, 0), IndirectObject(310, 0), IndirectObject(311, 0), IndirectObject(312, 0), IndirectObject(314, 0), IndirectObject(315, 0), IndirectObject(316, 0), IndirectObject(317, 0), IndirectObject(318, 0), IndirectObject(319, 0), IndirectObject(320, 0), IndirectObject(321, 0), IndirectObject(322, 0), IndirectObject(323, 0), IndirectObject(324, 0), IndirectObject(325, 0)],

False

In [92]:
import sys
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1
pdf_file_name = 'w8_ben.pdf'

fp = open(pdf_file_name, 'rb')

parser = PDFParser(fp)
doc = PDFDocument(parser)
fields = resolve1(doc.catalog['AcroForm'])['Fields']
for i in fields:
    field = resolve1(i)
    name, value = field.get('T'), field.get('V')
    print ('{0}: {1}'.format(name, value))

KeyError: 'AcroForm'

In [27]:

def extract_text_from_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    extracted_text = ''

    for img in images:
        img_text = pytesseract.image_to_string(img,config='--psm 12 --oem 3')
        extracted_text += img_text + '\n'

    return extracted_text

form_text=extract_text_from_pdf("w8_ben.pdf")
print (form_text)

rom W-8BEN

Certificate of Foreign Status of Beneficial Owner for United

States Tax Withholding and Reporting (Individuals)

(Rev. October 2021)

> For use by individuals. Entities must use Form W-8BEN-E.

OMB No. 1545-1621

Department of the Treasury

> Go to www.irs.gov/FormW8BEN for instructions and the latest information.

Internal Revenue Service

> Give this form to the withholding agent or payer. Do not send to the IRS.

Do NOT use this form if:

Instead, use Form:

* You are NOT an individual

W-8BEN-E

W-9

¢ You are a U.S. citizen or other U.S. person, including a resident alien individual

¢ You are a beneficial owner claiming that income is effectively connected with the conduct of trade or business within the United States

(other than personal services)

W-8ECI

¢ You are a beneficial owner who is receiving compensation for personal services performed in the United States

8233 or W-4

¢ You are a person acting as an intermediary

. W-8IMY

Note: If you are resident in a

In [94]:

from pdfminer.high_level import extract_text
# Read the PDF and extract text
def extract_text_from_pdf(pdf_path):
    text = extract_text(pdf_path)
    return text
form_text=extract_text_from_pdf("W8_ben.pdf")
print (form_text)


In [77]:


# from PyPDF2 import PdfReader
from fillpdf import fillpdfs
# from pdfrw import PdfReader
import re
import json
import PyPDF2
import pdfrw


def readpdf_fields1(pdf_file_name):
    f = PyPDF2.PdfReader(pdf_file_name)
    fields = f.get_fields()
    fdfinfo = dict((k, v.get('/V', '')) for k, v in fields.items())
    return fdfinfo
   
def readpdf_fields(pdf_file_name):
   
   fields=fillpdfs.get_form_fields(pdf_file_name,sort=False)
   return fields


def readpdf_fields3(pdf_file_name):
    pdf = pdfrw.PdfReader(pdf_file_name)
    # print(pdf.keys())
    # print(pdf.Info)
    # print(pdf.Root.keys())
    # field_index=1

    all_fields={}
    for page in pdf.pages:
        annotations = page['/Annots']
        if annotations is None:
            continue
        
        for annotation in annotations:
            if annotation['/Subtype']=='/Widget':
                if annotation['/T']:
                    key = annotation['/T'].to_unicode()
                    value = annotation['/V']
                    all_fields[key] = value
                   
    return all_fields             
        







In [78]:
pdf_file_name = 'input.pdf'

print (readpdf_fields(pdf_file_name),'\n\n')

print (readpdf_fields3(pdf_file_name),'\n\n')
print (readpdf_fields1(pdf_file_name),'\n\n')

{'FEFF0066005F0031005B0030005D': 'fdgfdg', 'FEFF0066005F0032005B0030005D': 'israel', 'FEFF0066005F0033005B0030005D': 'dfgfdg', 'FEFF0066005F0034005B0030005D': '1234 dereh pt', 'FEFF0066005F0035005B0030005D': 'israel', 'FEFF0066005F0036005B0030005D': 'fdgfdg', 'FEFF0066005F0037005B0030005D': 'dfgfdg', 'FEFF0066005F0038005B0030005D': '', 'FEFF0066005F0039005B0030005D': '1234-343', 'FEFF0066005F00310030005B0030005D': '123124 343', 'FEFF00630031005F00300031005B0030005D': '1', 'FEFF0066005F00310031005B0030005D': 'ewrwer', 'FEFF0066005F00310032005B0030005D': '12/12/12', 'FEFF0066005F00310033005B0030005D': '', 'FEFF0066005F00310034005B0030005D': 'src', 'FEFF0066005F00310035005B0030005D': '5', 'FEFF0066005F00310036005B0030005D': 'sdfsdf', 'FEFF0066005F00310037005B0030005D': '', 'FEFF0066005F00310038005B0030005D': '', 'FEFF00630031005F00300032005B0030005D': '1', 'FEFF0066005F00320030005B0030005D': '', 'FEFF0044006100740065005B0030005D': '', 'FEFF0066005F00320031005B0030005D': 'Diii sh'} 


{'f_

In [10]:
import re
import json


def get_all_values_w8_ben_form(pdf_text):
    # Define a dictionary to store extracted values
    extracted_values = {}

    # Define regular expressions for specific fields
    fields = {
        "Name of individual who is the beneficial owner": r'1\. Name of individual who is the beneficial owner\s*(.*)',
        "Country of citizenship": r'2\. Country of citizenship\s*(.*)',
        "Permanent Residence Address": r'3\. Permanent Residence Address\s*([\s\S]*?)4\.',
        "Mailing Address": r'4\. Mailing Address \(if different from above\)\s*([\s\S]*?)5\.',
        "U.S. taxpayer identification number (TIN)": r'5\. U\.S\. taxpayer identification number \(TIN\) or foreign tax identification number \(if required\)\s*(.*)',
        "Date of birth": r'6\. Date of birth \(MM-DD-YYYY\)\s*(.*)',
        "Country of birth": r'7\. Country of birth\s*(.*)',
        "Type of beneficial owner": r'8\. Type of beneficial owner\s*(.*)',
        "Special rates and conditions": r'9\. Special rates and conditions \(if applicable—see instructions\):\s*(.*)'
    }

    # Iterate through the defined fields and extract values
    for field, pattern in fields.items():
        match = re.search(pattern, pdf_text, re.DOTALL)
        if match:
            extracted_values[field] = match.group(1).strip()

    # Convert extracted values to JSON
    extracted_json = json.dumps(extracted_values, indent=4)


    return extracted_json
    # Print the JSON

print(get_all_values_w8_ben_form(form_text))


TypeError: cannot use a string pattern on a bytes-like object

In [87]:
import re
import fitz  # PyMuPDF

def identify_form_type(pdf_path):
    pdf_document = fitz.open(pdf_path)
    
    form_type = None
    
    for page_number in range(pdf_document.page_count):
        page = pdf_document.load_page(page_number)
        text = page.get_text("text")
        
        print(text)
        # Define keywords for each form type
        form_keywords = {
            "W-8BEN": ["W-8BEN", "foreign status"],
            "W-9": ["W-9", "Taxpayer Identification Number"],
            "W-8IN": ["W-8IN", "Chapter 4"]
        }
        
        for form_name, keywords in form_keywords.items():
            found_all_keywords = all(re.search(keyword, text, re.IGNORECASE) for keyword in keywords)
            if found_all_keywords:
                form_type = form_name
                break
            
    pdf_document.close()
    return form_type


# Provide the path to your PDF file
pdf_path = 'W8_ben.pdf'

# Call the function to identify form type
detected_form_type = identify_form_type(pdf_path)

if detected_form_type:
    print(f"Detected Form Type: {detected_form_type}")

    # Perform actions based on the detected form type
    if detected_form_type == "W-8BEN":
        # Handle W-8BEN form
        pass
    elif detected_form_type == "W-9":
        # Handle W-9 form
        pass
    elif detected_form_type == "W-8IN":
        # Handle W-8IN form
        
        pass
else:
    print("Could not determine the form type.")






 

Could not determine the form type.
